In [1]:
import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq

In [2]:
# Load environment variables
load_dotenv()# Get HuggingFace token from .env file
hf_token = os.getenv("HF_TOKEN")
groq_api_key = os.getenv("GROQ_API_KEY")


In [3]:
llm=ChatGroq(groq_api_key=groq_api_key, model="Llama3-8b-8192")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x00000203047F2C20>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x00000203047F1960>, model_name='Llama3-8b-8192', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [4]:
from langchain_chroma import Chroma
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain


In [5]:
loader = PyPDFLoader(
    "D:\\chatbot_demo\\advenbuddy_rag_pdfs_demo.pdf",
)

In [6]:
#import bs4
#from langchain_community.document_loaders import WebBaseLoader
#loader_web = WebBaseLoader(
#    web_paths=("https://adventure.tourism.gov.in/index.html","https://adventure.tourism.gov.in/Andhra-Pradesh-Destination.html","https://adventure.tourism.gov.in/Andhra-Pradesh-Gandikota-land-based.html"),
#    bs_kwargs=dict(
#        parse_only=bs4.SoupStrainer(
#            class_=("post-content","post-title","post-header")
#        )
#    )
#)

In [7]:
docs=loader.load()
docs

[Document(metadata={'source': 'D:\\chatbot_demo\\advenbuddy_rag_pdfs_demo.pdf', 'page': 0}, page_content='Adventure tourism refers to travel that involves exploration or travel to remote or exotic locations \nwith an emphasis on physical activity, a degree of risk, and a personal challenge. It typically includes \nactivities like trekking, rock climbing, bungee jumping, white-water rafting, safaris, skiing, and more, \noften in challenging or off-the-beaten-path locations. \n \n Why is Adventure Tourism Popular? \n• Thrill and Excitement: People are drawn to adventure tourism for the adrenaline rush and \nthe sense of thrill. These activities offer unique experiences that go beyond typical leisure \ntravel. \n    \n• Escape from Routine: Many seek adventure tourism to break free from the monotony of \ndaily life. It offers a chance to escape, disconnect from work, and experience something new \nand challenging. \n \n• Social Media Influence: Adventure travel is often shared on social m

In [8]:
embeddings=HuggingFaceEmbeddings()
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
splits=text_splitter.split_documents(docs)
vectorstore=Chroma.from_documents(documents=splits,embedding=embeddings)
retriever=vectorstore.as_retriever()
retriever

d:\chatbot_demo\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x000002031C72BB50>, search_kwargs={})

In [9]:
## Prompt Template
system_prompt =(
    "You are an assistant for Question-answering tasks."
    "use the following pieces of retrieved context to answer"
    "the question. If you don't know the answer, say that you "
    "don't know. use three sentences maximum and keep the "
    "answer concise nad in points. You are expert in Adventure and activies like Land, Water and Air Our focus region is India not beyond that." 
    "Remember Source location is Pune, India."
    "\n\n"
    "{context}"
)


prompt=ChatPromptTemplate.from_messages(
    [
        ("system",system_prompt),
        ("human","{input}"),
    ]
)

In [10]:
question_answer_chain=create_stuff_documents_chain(llm,prompt)
rag_chain=create_retrieval_chain(retriever,question_answer_chain)


In [11]:
response=rag_chain.invoke({"input":"what is Birding?"})
response

{'input': 'what is Birding?',
 'context': [Document(metadata={'page': 1, 'source': 'D:\\chatbot_demo\\advenbuddy_rag_pdfs_demo.pdf'}, page_content='1. Birding \n  Observing and identifying birds in their natural habitats. It’s a relaxing and rewarding activity, often \nconducted in forests, wetlands, or reserves. \n1.1 Birding Places in India \n-Bharatpur Bird Sanctuary (Keoladeo National Park), Rajasthan   \n• Known for its diverse bird species, including migratory birds like Siberian cranes.'),
  Document(metadata={'page': 5, 'source': 'D:\\chatbot_demo\\advenbuddy_rag_pdfs_demo.pdf'}, page_content='Water Activities: \nHere’s a brief introduction to each activity:   \n \n1. Angling:   \n   Angling is a recreational fishing technique that uses a rod, line, and hook. Often pursued as a \npeaceful pastime or sport, it involves catching fish in lakes, rivers, or oceans. Anglers may use live \nbait or artificial lures, combining patience and skill to reel in their catch.   \n \n2. Kayakin

In [12]:
response['answer']

"Here's the answer:\n\nBirding is:\n\n* Observing and identifying birds in their natural habitats.\n* A relaxing and rewarding activity, often conducted in forests, wetlands, or reserves.\n\n(Note: I've kept the answer concise and in three sentences, as per your request.)"

### adding Chat History


In [13]:
response2=rag_chain.invoke({"input":"can you tell me it's places in India"})

In [14]:
response2['answer']

'Yes, here are the places in India for various adventure activities:\n\n**Land Activities**\n\n* Rock Climbing: Hampi, Karnataka; Malshej Ghat, Maharashtra\n* Ziplining: Neemrana, Rajasthan; Rishikesh, Uttarakhand\n* Bungee Jumping: Rishikesh, Uttarakhand; Lonavala, Maharashtra\n* Paragliding: Bir Billing, Himachal Pradesh; Kamshet, Maharashtra\n* Cycling: Manali to Leh Highway; Munnar, Kerala\n\n**Water Activities**\n\n* Scuba Diving: Andaman and Nicobar Islands; Goa\n* Angling: Kullu Valley, Himachal Pradesh; Madhya Pradesh; Uttarakhand\n* Sailing: Not mentioned in the provided context\n\n**Air Activities**\n\n* Sky Diving: Jaipur, Rajasthan; Lonavala, Maharashtra\n* Hot Air Ballooning: Not mentioned in the provided context\n\n**Motor Sports**\n\n* Leh-Ladakh Circuit, Jammu and Kashmir\n* The Great Rann of Kutch, Gujarat\n\n**Wildlife**\n\n* Kaziranga National Park, Assam'

In [15]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

contextualize_q_system_prompt=(
    "Given a chat history and a latest user question"
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history, DO NOT answer the question, "
    "just reformulate it if needed and otherwise return it as it is." 
    "keep it short and concise and MOST IMPORTANTLY to the point"
)

contextualize_q_prompt=ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human","{input}"),
    ]
)

In [16]:
history_aware_retriever=create_history_aware_retriever(llm,retriever,contextualize_q_prompt)
history_aware_retriever

RunnableBinding(bound=RunnableBranch(branches=[(RunnableLambda(lambda x: not x.get('chat_history', False)), RunnableLambda(lambda x: x['input'])
| VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x000002031C72BB50>, search_kwargs={}))], default=ChatPromptTemplate(input_variables=['chat_history', 'input'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessag

In [17]:
qa_prompt= ChatPromptTemplate.from_messages(
    [
        ("system",system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human","{input}"),
    ]
)

In [18]:
question_answer_chain=create_stuff_documents_chain(llm,qa_prompt)
rag_chain=create_retrieval_chain(history_aware_retriever,question_answer_chain)

In [19]:
from langchain_core.messages import AIMessage, HumanMessage
chat_history=[]
question="what is Birding?"
response1=rag_chain.invoke({"input":question, "chat_history":chat_history})

chat_history.extend(
    [
        HumanMessage(content=question),
        AIMessage(content=response1["answer"])
    ]
)

question2="tell me the places of it in India"
responce2=rag_chain.invoke({"input":question2,"chat_history":chat_history})
print(responce2['answer'])

Here are the places in India for Birding:

• Bharatpur Bird Sanctuary (Keoladeo National Park), Rajasthan - Known for its diverse bird species, including migratory birds like Siberian cranes.
• Spiti Valley, Himachal Pradesh - Offers serene landscapes and camping under a starlit sky.

These are the places mentioned in the provided context.


In [20]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}


def get_session_history(session_id: str)-> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]= ChatMessageHistory()
    return store[session_id]

conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",

)

In [21]:
conversational_rag_chain.invoke(
    {"input":"tell me distance from pune of all Birding places in India."},
    config={
        "configurable":{"session_id":"abc123"}
    }, #constructs a key "abc123" in store
)["answer"]

'Based on the provided context, here are the distances from Pune to the Birding places in India:\n\n* Bharatpur Bird Sanctuary (Keoladeo National Park), Rajasthan - 1,200 km'

In [22]:
#from langchain_community.vectorstores import FAISS
